In [1]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.io import  gtfs_importer
from quetzal.model import stepmodel
import networkx as nx
from quetzal.engine import nested_logit
from imp import reload
reload(nested_logit)

<module 'quetzal.engine.nested_logit' from '../../../quetzal\\quetzal\\engine\\nested_logit.py'>

In [2]:
training_folder = '../../'

In [3]:
sm = stepmodel.read_zip(training_folder + r'model/transport/pt_pathfinder.zip')

read_hdf: : 100%|██████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 13.56it/s]


# assign a route_type to all paths

In [4]:
los = sm.pt_los
route_type_dict = sm.links['route_type'].to_dict()
los['route_types'] = los['link_path'].apply(lambda p: tuple({route_type_dict[l] for l in p}))
los['route_type'] = los['route_types'].apply(lambda s: max(s) if len(s) else 'walk')

# evaluate paths -> calculate utility

In [5]:
sm.pt_los['utility'] = - sm.pt_los['gtime'] / 1800

# pt_split

In [6]:
def step_pt_split(
    self,  
    verbose=False, 
    mode_nests=None, 
    nested_logit_structure=None
):
    
    r = nested_logit.nested_logit_from_paths(
        self.pt_los, 
        mode_nests=mode_nests,
        phi=nested_logit_structure,
        verbose=verbose
    )
    self.pt_los, self.pt_utilities, self.pt_probabilities = r

In [7]:
reload(nested_logit)

<module 'quetzal.engine.nested_logit' from '../../../quetzal\\quetzal\\engine\\nested_logit.py'>

In [10]:
mode_nests = {
    'root': ['pt', 'car', 'walk'],
    'pt':['massive','bus'],
    'massive':['subway', 'tram', 'rail']
}

nested_logit_structure = {
    'root': 1,
    'pt': 0.75,
    'car': 0.01, # los are obtained with wardrop equilibrium
    'walk': 0.75,
    'massive': 0.5,
    'subway': 0.5,
    'tram': 0.25,
    'rail': 0.25,
    'bus':0.25
}

In [11]:
sm.pt_los['utility'] = - sm.pt_los['gtime'] / 3600

In [12]:
step_pt_split(
    sm, 
    mode_nests=mode_nests,
    nested_logit_structure=nested_logit_structure, 
    verbose=True
)

path utilities bus 0.25 -> bus
path utilities rail 0.25 -> rail
path utilities subway 0.5 -> subway
path utilities tram 0.25 -> tram
path utilities walk 0.75 -> walk
mode utilities ['subway', 'tram', 'rail'] 0.5 -> massive
mode utilities ['massive', 'bus'] 0.75 -> pt
mode utilities ['pt', 'car', 'walk'] 1 -> root
mode probabilities root 1 -> ['pt', 'car', 'walk']
mode probabilities pt 0.75 -> ['massive', 'bus']
mode probabilities massive 0.5 -> ['subway', 'tram', 'rail']
path probabilities bus 0.25 -> bus
path probabilities rail 0.25 -> rail
path probabilities subway 0.5 -> subway
path probabilities tram 0.25 -> tram
path probabilities walk 0.75 -> walk


In [15]:
sm.pt_los

,alighting_links,alightings,all_walk,boarding_links,boardings,broken_modes,broken_route,destination,footpaths,gtime,...,path,pathfinder_session,reversed,time_link_path,transfers,route_types,route_type,utility,rank,probability
0,[],[],True,[],[],NaN,NaN,zone_0,[],0.000000,...,"(zone_0,)",best_path,False,0.0,[],(),walk,-0.000000,0,1.000000
1,[],[],True,[],[],NaN,NaN,zone_1,[],583.798520,...,"(zone_0, 115, zone_1)",best_path,False,0.0,[],(),walk,-0.162166,0,1.000000
2,[link_19685],[354],False,[link_19680],[440],NaN,NaN,zone_10,[],1127.765021,...,"(zone_0, 440, link_19680, link_19681, link_196...",best_path,False,420.0,[],"(bus,)",bus,-0.313268,0,0.367585
3,[link_15410],[354],False,[link_15409],[340],"{tram, bus, rail}",NaN,zone_10,"[(176, 340)]",1673.562267,...,"(zone_0, 176, 340, link_15409, link_15410, 354...",mode_braker,False,120.0,[],"(subway,)",subway,-0.464878,0,0.300308
4,[],[],True,[],[],"{subway, bus, tram, rail}",NaN,zone_10,"[(176, 340), (340, 88), (88, 490)]",2030.607094,...,"(zone_0, 176, 340, 88, 490, zone_10)",mode_braker,False,0.0,[],(),walk,-0.564058,0,0.332107
5,"[link_6984, link_7143, link_7697]","[392, 121, 324]",False,"[link_6982, link_7135, link_7692]","[440, 392, 121]","{subway, tram, rail}",NaN,zone_100,[],2761.220433,...,"(zone_0, 440, link_6982, link_6983, link_6984,...",mode_braker,False,1140.0,"[392, 121]","(bus,)",bus,-0.767006,0,0.302636
6,[link_9413],[53],False,[link_9413],[329],"{subway, bus, tram}",NaN,zone_100,"[(148, 457), (457, 329), (121, 356)]",4137.118837,...,"(zone_0, 148, 457, 329, link_9413, 53, zone_26...",mode_braker,False,240.0,[],"(rail,)",rail,-1.149200,0,0.110142
7,[link_9929],[324],False,[link_9917],[115],NaN,NaN,zone_100,[],1883.479507,...,"(zone_0, 115, link_9917, link_9918, link_9919,...",best_path,False,960.0,[],"(subway,)",subway,-0.523189,0,0.385212
8,[],[],True,[],[],"{subway, bus, tram, rail}",NaN,zone_100,"[(90, 224), (121, 356)]",5089.016797,...,"(zone_0, 115, zone_1, 237, zone_3, 90, 224, zo...",mode_braker,False,0.0,[],(),walk,-1.413616,0,0.202009
9,[link_9288],[499],False,[link_9272],[148],"{subway, tram, rail}",NaN,zone_101,[],2373.914485,...,"(zone_0, 148, link_9272, link_9273, link_9274,...",mode_braker,False,1260.0,[],"(bus,)",bus,-0.659421,0,0.335174


In [13]:
sm.pt_probabilities

route_type,origin,destination,root,walk,car,pt,bus,massive,rail,tram,subway
0,zone_0,zone_0,1,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,zone_0,zone_1,1,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,zone_0,zone_10,1,0.332107,0.0,0.667893,0.367585,0.300308,0.000000,0.000000,0.300308
3,zone_0,zone_100,1,0.202009,0.0,0.797991,0.302636,0.495355,0.110142,0.000000,0.385212
4,zone_0,zone_101,1,0.215814,0.0,0.784186,0.335174,0.449013,0.116232,0.000000,0.332781
5,zone_0,zone_102,1,0.183459,0.0,0.816541,0.358568,0.457973,0.093399,0.000000,0.364574
6,zone_0,zone_103,1,0.178713,0.0,0.821287,0.339767,0.481520,0.209594,0.000000,0.271926
7,zone_0,zone_104,1,0.176167,0.0,0.823833,0.326752,0.497081,0.072382,0.116472,0.308227
8,zone_0,zone_105,1,0.214778,0.0,0.785222,0.342108,0.443114,0.000000,0.086402,0.356712
9,zone_0,zone_106,1,0.256369,0.0,0.743631,0.350082,0.393550,0.000000,0.000000,0.393550


In [63]:
sm.pt_probabilities.mean().T

route_type
root       1.000000
walk       0.235334
car        0.000000
pt         0.764666
bus        0.299761
massive    0.464905
rail       0.083222
tram       0.051201
subway     0.330481
dtype: float64

In [64]:
sm.to_zip(training_folder + 'model/transport/pt_split.zip')

to_hdf(new file): 100%|████████████████████████████████████████████████████████████████| 33/33 [00:04<00:00,  6.77it/s]
